In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

from google.colab import drive
drive.mount('/content/drive')


base_dir = "/content/drive/My Drive/labsfile/"

happy_dir = os.path.join(base_dir, "happy/")
sad_dir = os.path.join(base_dir, "sad/")

# Визначення зворотного виклику
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') is not None and logs.get('accuracy') > 0.999:
            print("\\nДосягнуто 99,9% точності, тому тренування скасовано!")
            self.model.stop_training = True

# Попередня обробка даних
def image_generator():
    # Створення об'єкта ImageDataGenerator
    train_datagen = ImageDataGenerator(rescale=1./255)

    # Завантаження зображень з директорії
    train_generator = train_datagen.flow_from_directory(base_dir,
                                                        target_size=(150, 150),
                                                        batch_size=10,
                                                        class_mode='binary')
    return train_generator

# Створення та навчання моделі
def train_happy_sad_model(train_generator):
    # Створення об'єкта зворотного виклику
    callbacks = myCallback()

    # Визначення моделі
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    # Компіляція моделі
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    # Навчання моделі
    history = model.fit(train_generator,
                        epochs=15,
                        callbacks=[callbacks])

    return history

# Виклик функцій для навчання моделі
train_generator = image_generator()
history = train_happy_sad_model(train_generator)
